<a href="https://colab.research.google.com/github/Vlasovets/MB-GAN/blob/master/MBGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/zhanxw/MB-GAN.git

In [ ]:
%cd MB-GAN/

In [9]:
import pickle
import os
from model import *
from utils import *
from mbgan_train_demo import *

from keras.layers import Input

In [10]:
FILE = "./data/raw_data.pkl"

data_o_case, data_o_ctrl, taxa_list = load_sample_pickle_data(FILE)

adj_matrix, taxa_indices = expand_phylo(taxa_list)

tf_matrix = adjmatrix_to_dense(adj_matrix, shape=(len(taxa_list), len(taxa_indices)))

In [11]:
model_config = {
        'ntaxa': 719,
        'latent_dim': 100,
        'generator': {'n_channels': 512},
        'critic': {'n_channels': 256, 'dropout_rate': 0.25, 
                   'tf_matrix': tf_matrix, 't_pow': 1000.}
    }
    
train_config = {
    'generator': {'optimizer': ('RMSprop', {}), 'lr': 0.00005},
    'critic': {'loss_weights': [1, 1, 10], 
                'optimizer': ('RMSprop', {}), 'lr': 0.00005},
}

In [12]:
NAME = "mbgan_case"
EXP_DIR = "NielsenHB_2014_stool"

mbgan = MBGAN(NAME, model_config, train_config)
mbgan.__dict__

{'critic': <keras.engine.functional.Functional at 0x7f52566b6250>,
 'critic_graph': <keras.engine.functional.Functional at 0x7f52d4a395d0>,
 'generator': <keras.engine.functional.Functional at 0x7f52566d9bd0>,
 'generator_graph': <keras.engine.functional.Functional at 0x7f52d49c9890>,
 'latent_dim': 100,
 'model_config': {'critic': {'dropout_rate': 0.25,
   'n_channels': 256,
   't_pow': 1000.0,
   'tf_matrix': array([[1., 0., 0., ..., 0., 0., 0.],
          [0., 1., 0., ..., 0., 0., 0.],
          [0., 0., 1., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 1., 1., 1.]])},
  'generator': {'n_channels': 512},
  'latent_dim': 100,
  'ntaxa': 719},
 'model_name': 'mbgan_case',
 'ntaxa': 719,
 'train_config': {'critic': {'loss_weights': [1, 1, 10],
   'lr': 5e-05,
   'optimizer': ('RMSprop', {})},
  'generator': {'lr': 5e-05, 'optimizer': ('RMSprop', {})}}}

In [13]:
ITERS = 500000
BATCH_SIZE = 32
SAVE_INTERVAL = 1000

mbgan.train(data_o_case, iteration=ITERS, batch_size=BATCH_SIZE, 
                n_critic=5, n_generator=1, save_fn=get_save_fn(taxa_list), 
                save_interval=SAVE_INTERVAL, experiment_dir=EXP_DIR, 
            pre_processor=None, verbose=0)

#####################################################
Training start at: 2022-06-08 13:42:22
Run MB-GAN for 500000 iterations with batch_size=32
Save generated samples and model every 1000 iters
Results are exported to folder: NielsenHB_2014_stool/mbgan_case_20220608T134222
    Create log folder: NielsenHB_2014_stool/mbgan_case_20220608T134222
    Create model folder: NielsenHB_2014_stool/mbgan_case_20220608T134222/models
Generator structure:
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 100)]             0         
                                                                 
 sequential_5 (Sequential)   (None, 719)               952015    
                                                                 
Total params: 952,015
Trainable params: 948,943
Non-trainable params: 3,072
________________________________________________________________

ValueError: ignored

In [ ]:
real = data_o_case[np.random.randint(0, data_o_case.shape[0], BATCH_SIZE)]

noise = np.random.normal(0, 1, (BATCH_SIZE, model_config['latent_dim']))

valid = -np.ones((BATCH_SIZE, 1))
fake =  np.ones((BATCH_SIZE, 1))
dummy = np.zeros((BATCH_SIZE, 1))


z = Input(shape=(model_config['latent_dim'],))
fake_sample = generator(z)
valid = critic(real_sample)
fake = critic(fake_sample)

critic_graph = Model(inputs=[real_sample, z], 
                     outputs=[valid, fake, validity_interpolated])

# generator_graph = Model(z, valid)

# d_loss = critic_graph.train_on_batch([real, noise], [valid, fake, dummy])
# critic_graph.train_on_batch([real, noise], [valid, fake, dummy])